In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [3]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Valuation&agg=Accumulated"

In [4]:

for year in years:
    with open("index_data_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)
            


    

In [5]:
dfs = []
for year in years :
    with open ("index_data_leaders_htmls/{}.html".format(year),encoding="utf-8") as f :
        page =f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr',class_="StatsHeaderGridResults")
        statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")
        index_data= pd.read_html(str(statistics_table))[0]
        index_data["Year"] = year
        dfs.append(index_data)
                

In [6]:

index_data = pd.concat(dfs)
index_data

,Rank,Player,Team,GamesG,Index RatingPIR,Ave.,Per 40 min.40m,Year
0,1,"BODIROGA, DEJANBODIROGA, D.",PAOPanathinaikos AthensPanathinaikos Athens,22,509,23.14,28.66,2001
1,2,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,469,23.45,26.62,2001
2,3,"AVLEEV, RUSLANAVLEEV, R.",PEMUral GreatUral Great,20,452,22.60,30.25,2001
3,4,"TURKCAN, MIRSADTURKCAN, M.",CSKCSKA MoscowCSKA Moscow,17,439,25.82,29.41,2001
4,5,"FUCKA, GREGORFUCKA, G.",FORSkipper BolognaSkipper Bologna,20,428,21.40,26.82,2001
...,...,...,...,...,...,...,...,...
45,44,"GIEDRAITIS, ROKASGIEDRAITIS, R.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,29,343,11.83,16.45,2021
46,47,"ENOCH, STEVENENOCH, S.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,32,342,10.69,20.98,2021
47,48,"SHIELDS, SHAVONSHIELDS, S.",AXMAX Armani Exchange MilanAX Armani Exchange ...,24,340,14.17,19.01,2021
48,49,"EVANS, KEENANEVANS, K.",MTAMaccabi Playtika Tel AvivMaccabi Playtika T...,33,338,10.24,16.25,2021


In [12]:
engine = create_engine('sqlite:///index_data.db')
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
index_data.to_sql(
    name = "index_data_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [14]:
index_data_select = session.execute('select * from "index_data_leaders"')
index_data_select.all()

[(0, 1, 'BODIROGA, DEJANBODIROGA, D.', 'PAOPanathinaikos AthensPanathinaikos Athens', 22, 509, 23.14, 28.66, 2001),
 (1, 2, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 469, 23.45, 26.62, 2001),
 (2, 3, 'AVLEEV, RUSLANAVLEEV, R.', 'PEMUral GreatUral Great', 20, 452, 22.6, 30.25, 2001),
 (3, 4, 'TURKCAN, MIRSADTURKCAN, M.', 'CSKCSKA MoscowCSKA Moscow', 17, 439, 25.82, 29.41, 2001),
 (4, 5, 'FUCKA, GREGORFUCKA, G.', 'FORSkipper BolognaSkipper Bologna', 20, 428, 21.4, 26.82, 2001),
 (5, 6, 'TOMASEVIC, DEJANTOMASEVIC, D.', 'BKNTau CeramicaTau Ceramica', 20, 394, 19.7, 27.76, 2001),
 (6, 7, 'EDNEY, TYUSEDNEY, T.', 'TREBenetton BasketBenetton Basket', 19, 385, 20.26, 26.72, 2001),
 (7, 8, 'GINOBILI, EMANUELGINOBILI, E.', 'VIRKinder Virtus BolognaKinder Virtus Bologna', 22, 376, 17.09, 24.01, 2001),
 (8, 9, 'GIRICEK, GORDANGIRICEK, G.', 'CSKCSKA MoscowCSKA Moscow', 18, 370, 20.56, 23.97, 2001),
 (9, 10, 'KAKIOUZIS, MICHAILISKAKIOUZIS, M.', 'AEKAEK AthensAEK Athens', 20, 363, 18.15